In [165]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
sns.set()

import sigma_equation as se

In [166]:
subjects = ["soggetto_1","soggetto_2","soggetto_3","soggetto_6","soggetto_7","soggetto_8"]

In [167]:
df_o_dict = {}
for subject in subjects:
    df_o_dict[subject] = pd.read_csv("Datasets/{}_weights_o.csv".format(subject))
    df_o_dict[subject].columns = ["TrRi","RiTr","Plu","Ttra","Ttrb","Tr","MeTr","MoTr","RiRi","MoFo","Mmof","MoRi","MeRi","Ri","Comp"]
    df_o_dict[subject].index = df_o_dict[subject].columns

In [168]:
# plot heatmaps with plotly express
for subject in subjects:
    fig = px.imshow(df_o_dict[subject],
            color_continuous_midpoint=0,
            color_continuous_scale = 'RdBu_r',
            zmin=-5, zmax=5,
            text_auto=".2",
            aspect="Auto",
            title="Original data for {}".format(subject))
    fig.show()

In [169]:
# calculate standard deviations (on axis 1) for each subject
stds = []
for subject in subjects:
    stds.append(df_o_dict[subject].std(axis=1))
stds = pd.DataFrame(stds)
stds.index = subjects
stds.columns = df_o_dict[subject].columns
stds

,TrRi,RiTr,Plu,Ttra,Ttrb,Tr,MeTr,MoTr,RiRi,MoFo,Mmof,MoRi,MeRi,Ri,Comp
soggetto_1,0.220196,0.223144,0.333889,0.437770,0.213160,0.228997,0.284814,0.177968,0.236333,0.214411,0.185073,0.297189,0.253959,0.377430,0.252651
soggetto_2,0.492646,0.363330,0.517538,0.312095,0.385892,0.122184,0.389117,0.385330,0.465440,0.258724,0.403519,0.264124,0.383842,0.433798,0.425059
soggetto_3,1.290731,0.798684,0.966400,0.715229,1.453136,0.661781,0.650314,0.544598,0.330313,0.710032,0.812042,0.894647,0.405209,0.501343,1.096286
soggetto_6,0.634487,0.959705,0.746840,1.216884,0.605079,0.922659,0.977046,1.284632,0.683250,0.809403,1.079089,0.853820,0.713823,1.024133,0.730252
soggetto_7,0.562738,0.313070,1.146880,0.845131,0.751379,0.673072,1.289125,0.797708,2.019221,1.006056,0.965304,0.732277,0.971913,0.849081,0.537363
soggetto_8,2.040547,1.172403,1.327796,0.890147,0.822927,1.342752,2.026490,0.988268,1.101534,0.706036,1.042398,1.864858,0.768913,1.271345,0.800889


In [170]:
# plot stds with barplot with plotly express
fig = px.bar(stds, barmode="group", title="Standard deviations for each subject")
# use different colors for each subject
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]
# ...
fig.show()

In [171]:
# for each subject's dataframe create two numpy arrays, one with positive values and one for negatives
df_ecc_dict = {}
df_ini_dict = {}
for subject in subjects:
    df_ecc_dict[subject] = df_o_dict[subject].where(df_o_dict[subject] > 0)
    df_ini_dict[subject] = df_o_dict[subject].where(df_o_dict[subject] < 0)
    # put the other values to zeros
    df_ecc_dict[subject] = df_ecc_dict[subject].fillna(0)
    df_ini_dict[subject] = df_ini_dict[subject].fillna(0)
    # transform the negative values of df_n_dict[subject] in positives
    df_ini_dict[subject] = df_ini_dict[subject] * (-1)
    df_ini_dict[subject].replace(-0.0, 0.0)
    # trasform each dataframe in a stochastic matrix by column (the sum of each column must be equal to 1)
    df_ecc_dict[subject] = df_ecc_dict[subject].div(df_ecc_dict[subject].sum(axis=0), axis=1)
    df_ini_dict[subject] = df_ini_dict[subject].div(df_ini_dict[subject].sum(axis=0), axis=1)

In [172]:
df_ecc_dict["soggetto_8"]

,TrRi,RiTr,Plu,Ttra,Ttrb,Tr,MeTr,MoTr,RiRi,MoFo,Mmof,MoRi,MeRi,Ri,Comp
TrRi,0.000000,0.000000,0.000000,0.002004,0.000000,0.271259,0.000000,0.000000,0.372986,0.000000,0.050735,0.031767,0.446254,0.000000,0.000000
RiTr,0.141128,0.095657,0.000000,0.018439,0.128261,0.238605,0.000000,0.000000,0.097695,0.000000,0.000000,0.030942,0.092450,0.000000,0.000000
Plu,0.058132,0.068976,0.092523,0.057709,0.149024,0.000000,0.223900,0.436567,0.000000,0.147726,0.118357,0.112787,0.000000,0.109793,0.218809
Ttra,0.021808,0.000000,0.146434,0.154363,0.000000,0.000000,0.001661,0.000000,0.081475,0.104693,0.000000,0.000000,0.000000,0.135440,0.000000
Ttrb,0.134501,0.158305,0.000000,0.136830,0.000000,0.000000,0.025526,0.000000,0.092211,0.004713,0.000000,0.000000,0.100985,0.006040,0.000000
Tr,0.000000,0.000000,0.104362,0.000000,0.000000,0.000000,0.173081,0.000000,0.080206,0.000000,0.207985,0.214334,0.088960,0.129828,0.000000
MeTr,0.000000,0.136253,0.038654,0.000000,0.000000,0.283747,0.131623,0.032945,0.090388,0.000000,0.365812,0.385657,0.144615,0.081717,0.047145
MoTr,0.153661,0.107431,0.182333,0.040593,0.186842,0.000000,0.025386,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.139389,0.000000
RiRi,0.120607,0.000000,0.000000,0.128676,0.339514,0.000000,0.000000,0.000000,0.005038,0.000000,0.000000,0.000000,0.007349,0.000000,0.000000
MoFo,0.055389,0.021136,0.000000,0.039320,0.000000,0.206389,0.000000,0.000000,0.092470,0.000000,0.000000,0.000000,0.049632,0.000000,0.192780


In [173]:
df_ini_dict["soggetto_8"]

,TrRi,RiTr,Plu,Ttra,Ttrb,Tr,MeTr,MoTr,RiRi,MoFo,Mmof,MoRi,MeRi,Ri,Comp
TrRi,0.080844,0.332993,0.424472,-0.000000,0.113205,-0.000000,0.132426,0.218278,-0.000000,0.318139,-0.000000,-0.000000,-0.000000,0.373705,0.241310
RiTr,-0.000000,-0.000000,0.214017,-0.000000,-0.000000,-0.000000,0.241481,0.259908,-0.000000,0.195290,0.102233,-0.000000,-0.000000,0.228367,0.136730
Plu,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.243506,-0.000000,-0.000000,0.288834,-0.000000,-0.000000,-0.000000,0.236467,-0.000000,-0.000000
Ttra,-0.000000,0.220787,-0.000000,-0.000000,0.031138,0.033954,-0.000000,0.034938,-0.000000,-0.000000,0.112870,0.099014,0.026083,-0.000000,0.046200
Ttrb,-0.000000,-0.000000,0.029749,-0.000000,0.048473,0.001246,-0.000000,0.114503,-0.000000,-0.000000,0.115412,0.227706,-0.000000,-0.000000,0.028342
Tr,0.247228,0.188595,-0.000000,0.181170,0.182340,0.027231,-0.000000,0.036451,-0.000000,0.054723,-0.000000,-0.000000,-0.000000,-0.000000,0.108726
MeTr,0.311305,-0.000000,-0.000000,0.422096,0.226305,-0.000000,-0.000000,-0.000000,-0.000000,0.028004,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
MoTr,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.097876,-0.000000,0.086769,0.062489,0.008755,0.172731,0.203445,0.160931,-0.000000,0.125239
RiRi,-0.000000,0.111455,0.162507,-0.000000,-0.000000,0.051430,0.073699,0.046900,-0.000000,0.216265,0.163454,0.104556,-0.000000,0.246156,0.159539
MoFo,-0.000000,-0.000000,0.085872,-0.000000,0.027146,-0.000000,0.047712,0.116453,-0.000000,0.058818,0.102899,0.135420,-0.000000,0.089964,-0.000000


In [174]:
# create dicts with numpy instead dataframes
np_ecc_dict = {}
np_ini_dict = {}
for subject in subjects:
    np_ecc_dict[subject] = df_ecc_dict[subject].to_numpy()
    np_ini_dict[subject] = df_ini_dict[subject].to_numpy()

In [175]:
np_ecc_dict["soggetto_8"]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.00372067e-03,
        0.00000000e+00, 2.71258693e-01, 0.00000000e+00, 0.00000000e+00,
        3.72985754e-01, 0.00000000e+00, 5.07350981e-02, 3.17673061e-02,
        4.46253553e-01, 0.00000000e+00, 0.00000000e+00],
       [1.41127702e-01, 9.56574629e-02, 0.00000000e+00, 1.84392118e-02,
        1.28261011e-01, 2.38605192e-01, 0.00000000e+00, 0.00000000e+00,
        9.76952834e-02, 0.00000000e+00, 0.00000000e+00, 3.09416326e-02,
        9.24496423e-02, 0.00000000e+00, 0.00000000e+00],
       [5.81321383e-02, 6.89759544e-02, 9.25227703e-02, 5.77092369e-02,
        1.49024071e-01, 0.00000000e+00, 2.23899554e-01, 4.36567336e-01,
        0.00000000e+00, 1.47725844e-01, 1.18356689e-01, 1.12786713e-01,
        0.00000000e+00, 1.09793486e-01, 2.18809137e-01],
       [2.18078764e-02, 0.00000000e+00, 1.46434419e-01, 1.54363281e-01,
        0.00000000e+00, 0.00000000e+00, 1.66052794e-03, 0.00000000e+00,
        8.14748202e-02, 1.04693379e-0

In [176]:
np_ini_dict["soggetto_1"]

array([[ 0.34469149,  0.41849418,  0.00206384,  0.44488317,  0.13419976,
         0.20570888,  0.09893122,  0.3130637 ,  0.24263741,  0.19528605,
         0.35750606,  0.17114488,  0.049462  ,  0.27305392,  0.091681  ],
       [ 0.11081296,  0.11898522,  0.17004562,  0.42322021,  0.01691697,
         0.17044478,  0.01676706,  0.10130866, -0.        ,  0.17746898,
        -0.        ,  0.0302237 ,  0.12269472,  0.1932612 , -0.        ],
       [-0.        , -0.        ,  0.17733637, -0.        ,  0.23934598,
         0.02058978,  0.25228392, -0.        , -0.        , -0.        ,
        -0.        ,  0.08633342, -0.        , -0.        ,  0.14312379],
       [-0.        , -0.        , -0.        , -0.        ,  0.12569915,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        ,  0.13181477,  0.01339112,  0.31729936],
       [-0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        , 

In [177]:
df_evolve_dict = {}
for subject in subjects:
    df_evolve_dict[subject] = pd.DataFrame(columns=df_o_dict[subject].columns)
for subject in subjects:
    for i in range(0,100):
        np_ecc_dict[subject] = np.matmul(np_ecc_dict[subject], np_ecc_dict[subject])
        np_ini_dict[subject] = np.matmul(np_ini_dict[subject], np_ini_dict[subject])
        print("SOGGETTO : " + subject)
        print("Eccitatorio")
        print(np.sum(np_ecc_dict[subject], axis=0))
        print("Inibitorio")
        print(np.sum(np_ini_dict[subject], axis=0))
        np_activations = np.sum(np_ecc_dict[subject], axis=0) - np.sum(np_ini_dict[subject], axis=0)
        np_activations = np_activations.reshape(15,1)
        #print(np_activations)
        df_evolve_dict[subject] = pd.concat(
            [
                df_evolve_dict[subject],
                pd.DataFrame(np_activations.T, columns = df_evolve_dict[subject].columns)
            ]
            , axis = 0)
    

/tmp/ipykernel_14253/2287471045.py:7: RuntimeWarning:

overflow encountered in matmul

/tmp/ipykernel_14253/2287471045.py:7: RuntimeWarning:

overflow encountered in matmul

/tmp/ipykernel_14253/2287471045.py:7: RuntimeWarning:

overflow encountered in matmul

/tmp/ipykernel_14253/2287471045.py:7: RuntimeWarning:

overflow encountered in matmul



In [178]:
# for each subject's dataframe adjust the index to start from 0
for subject in subjects:
    df_evolve_dict[subject].index = range(0,df_evolve_dict[subject].shape[0])

In [179]:
df_evolve_dict["soggetto_8"]

,TrRi,RiTr,Plu,Ttra,Ttrb,Tr,MeTr,MoTr,RiRi,MoFo,Mmof,MoRi,MeRi,Ri,Comp
0,0.000000e+00,-2.220446e-16,0.000000e+00,0.000000e+00,1.110223e-16,-4.440892e-16,-1.110223e-16,1.110223e-16,0.000000e+00,0.000000e+00,-4.440892e-16,-4.440892e-16,1.110223e-16,2.220446e-16,-1.110223e-16
1,1.110223e-16,-2.220446e-16,2.220446e-16,-1.110223e-16,1.110223e-16,-2.220446e-16,-2.220446e-16,1.110223e-16,0.000000e+00,0.000000e+00,-4.440892e-16,-2.220446e-16,3.330669e-16,-1.110223e-16,-3.330669e-16
2,-3.330669e-16,-2.220446e-16,-2.220446e-16,-1.110223e-16,4.440892e-16,-2.220446e-16,-4.440892e-16,0.000000e+00,0.000000e+00,0.000000e+00,-2.220446e-16,-3.330669e-16,0.000000e+00,1.110223e-16,-2.220446e-16
3,-2.220446e-16,-7.771561e-16,-4.440892e-16,-3.330669e-16,0.000000e+00,-4.440892e-16,-2.220446e-16,0.000000e+00,-3.330669e-16,-2.220446e-16,-4.440892e-16,-7.771561e-16,-4.440892e-16,-3.330669e-16,0.000000e+00
4,-5.551115e-16,-9.992007e-16,-4.440892e-16,-7.771561e-16,-4.440892e-16,-9.992007e-16,-7.771561e-16,-5.551115e-16,-5.551115e-16,-7.771561e-16,-8.881784e-16,-7.771561e-16,-5.551115e-16,-6.661338e-16,-4.440892e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
96,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
97,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
98,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf


In [180]:
# plot lineplots with plotly express for each subject
for subject in subjects:
    fig = px.line(df_evolve_dict[subject], title="Evolution of activations for {}".format(subject))
    fig.show()